In [103]:
import requests
import json
import gzip
from tqdm import tqdm

In [104]:
# 10K patients
dataset='10K_patients'
user_id='1f4575c9-80ea-425c-9d3e-c2b79e9159da'
client_secret='e5d8a82f7e9e3b1d1a6aff39333b7d64a03c0aed76fa75b406bd0c52b9fdbe69ee2cea511275a498'

# 100 patients
# dataset='100_patients'
# user_id='e75679c2-1b58-4cf5-8664-d3706de8caf5'
# client_secret='5a2dff22afeb43ebfeedb1e510ac42b3891d4abef8a1ff7129a153344458b7834bef4adb6deb028a'


In [105]:
# Get an access token
r = requests.post('https://sandbox.bcda.cms.gov/auth/token', auth=(user_id, client_secret), headers={'accept': 'application/json'})
assert(r.status_code==200)
response = r.json()
access_token = response['access_token']

In [106]:
# Submit the job request
r = requests.get('https://sandbox.bcda.cms.gov/api/v1/Patient/$export', 
                headers={'accept': 'application/fhir+json',
                        'Prefer': 'respond-async',
                        'Authorization': f'Bearer {access_token}'})
assert(r.status_code==202)
content_location = r.headers['Content-Location']

In [125]:
# Check job status
r = requests.get(content_location, 
                 headers={'accept': 'application/fhir+json',
                          'Authorization': f'Bearer {access_token}'})

if r.status_code == 200:
    print('Done!')
elif r.status_code == 202:
    print(r.headers['X-Progress'])
else:
    print('ERROR')

Pending


In [121]:
# Download files
response = r.json()['output']
for download in tqdm(response): 
    r = requests.get(download['url'], 
                     headers={'Accept-Encoding': 'gzip',
                              'Authorization': f'Bearer {access_token}'})

    object_type=download['type']
    with gzip.open(f'{dataset}_{object_type}.ndjson.gz', 'wb') as f:
        f.write(r.content)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)